<a href="https://colab.research.google.com/github/thegregbeyond/FreeFuse-AI-Calbright-Project/blob/main/Image_Detection_and_Visualization_Script.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Step 1: (Uncomment to install if needed)
# !pip install pandas opencv-python-headless google-colab -q

import cv2
import pandas as pd
from pathlib import Path
from typing import Optional
from google.colab import drive

# ─── User-Configurable Parameters ─────────────────────────────────────────────
INPUT_CSV     = Path("/content/drive/MyDrive/FreeFuse_Project/Extracted_Stills/YOLOv10/final_annotations.csv")
STILLS_FOLDER = Path("/content/drive/MyDrive/FreeFuse_Project/Extracted_Stills/YOLOv10")
OUTPUT_FOLDER = STILLS_FOLDER / "Visualized_Annotations"

# Drawing settings
BOX_COLOR         = (0, 255, 0)   # BGR
TEXT_COLOR        = (255, 255, 255)
CENTER_DOT_COLOR  = (0, 0, 255)
BOX_THICKNESS     = 2
CENTER_DOT_RADIUS = 5
FONT_FACE         = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE        = 0.5
# ────────────────────────────────────────────────────────────────────────────────

def mount_drive():
    print("🔗 Mounting Google Drive…")
    drive.mount("/content/drive", force_remount=True)
    print("✅ Drive mounted.")

def load_annotations(csv_path: Path) -> Optional[pd.DataFrame]:
    if not csv_path.exists():
        print(f"❌ Annotation file not found: {csv_path}")
        return None
    df = pd.read_csv(csv_path)
    print(f"✅ Loaded {len(df)} annotations from {csv_path.name}")
    return df

def visualize_annotations_for_image(image_file: str, df: pd.DataFrame):
    annotations = df[df["image_file_name"] == image_file]
    if annotations.empty:
        print(f"⚠️  No annotations for {image_file}, skipping.")
        return

    img_path = STILLS_FOLDER / image_file
    if not img_path.exists():
        print(f"❌ Still image not found: {img_path}")
        return

    # Load image
    img = cv2.imread(str(img_path))
    if img is None:
        print(f"❌ Could not load image: {img_path}")
        return

    # Draw each annotation
    for _, row in annotations.iterrows():
        x1, y1 = int(row["x_min"]), int(row["y_min"])
        x2, y2 = int(row["x_max"]), int(row["y_max"])
        label   = str(row["object_name"])

        # Bounding box
        cv2.rectangle(img, (x1, y1), (x2, y2), BOX_COLOR, BOX_THICKNESS)

        # Center dot
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        cv2.circle(img, (cx, cy), CENTER_DOT_RADIUS, CENTER_DOT_COLOR, -1)

        # Label background
        (w, h), _ = cv2.getTextSize(label, FONT_FACE, FONT_SCALE, 1)
        cv2.rectangle(img, (x1, y1 - h - 4), (x1 + w, y1), BOX_COLOR, -1)

        # Label text
        cv2.putText(img, label, (x1, y1 - 2), FONT_FACE, FONT_SCALE, TEXT_COLOR, 1, cv2.LINE_AA)

    # Save the visualization
    out_path = OUTPUT_FOLDER / f"{Path(image_file).stem}_visualized.jpg"
    out_path.parent.mkdir(parents=True, exist_ok=True)
    cv2.imwrite(str(out_path), img)
    print(f"✅ Saved visualization: {out_path.name}")

def main():
    mount_drive()
    df = load_annotations(INPUT_CSV)
    if df is None:
        return

    # Make sure output folder exists
    OUTPUT_FOLDER.mkdir(parents=True, exist_ok=True)

    # Iterate over every unique still in the annotations
    for image_file in df["image_file_name"].unique():
        visualize_annotations_for_image(image_file, df)

    print("\n🎉 All stills processed!")

if __name__ == "__main__":
    main()

🔗 Mounting Google Drive…
Mounted at /content/drive
✅ Drive mounted.
✅ Loaded 888 annotations from final_annotations.csv
✅ Saved visualization: 1086533-hd_1280_720_25fps_0001_visualized.jpg
✅ Saved visualization: 1197801-hd_1920_1080_25fps_0001_visualized.jpg
✅ Saved visualization: 1197801-hd_1920_1080_25fps_0004_visualized.jpg
✅ Saved visualization: 1197802-hd_1920_1080_25fps_0001_visualized.jpg
✅ Saved visualization: 1322056-hd_1920_1080_30fps_0001_visualized.jpg
✅ Saved visualization: 13856488_3840_2160_60fps_0001_visualized.jpg
✅ Saved visualization: 1509518-uhd_3840_2160_30fps_0001_visualized.jpg
✅ Saved visualization: 1562544-sd_568_320_30fps_0001_visualized.jpg
✅ Saved visualization: 1580509-hd_1920_1080_30fps_0001_visualized.jpg
✅ Saved visualization: 2103099-uhd_3840_2160_30fps_0001_visualized.jpg
✅ Saved visualization: 2281947-hd_1920_1080_25fps_0007_visualized.jpg
✅ Saved visualization: 2281947-hd_1920_1080_25fps_0025_visualized.jpg
✅ Saved visualization: 2317680-uhd_3840_216